<a href="https://colab.research.google.com/github/zee404-code/DartsTimeSeries/blob/main/dart08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install darts

In [55]:
from darts.datasets import AirPassengersDataset, AusBeerDataset, MonthlyMilkDataset, HeartRateDataset     
from darts.utils.statistics import check_seasonality, extract_trend_and_seasonality,remove_seasonality, remove_trend, stationarity_test_adf, stationarity_test_kpss, stationarity_tests
from darts.utils.model_selection import train_test_split
from darts.utils.utils import ModelMode, SeasonalityMode
from darts.models import AutoARIMA, ARIMA, VARIMA
from darts.metrics import mape, coefficient_of_variation, dtw_metric, mae, marre, mase, mse, ope, r2_score, rho_risk, rmse, rmsle, smape

In [ ]:
data1 = AirPassengersDataset().load()
data1.plot()
train1, test1 = train_test_split(data1, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)

In [ ]:
data2 = AusBeerDataset().load()
data2.plot()
train2, test2 = train_test_split(data2, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)

In [ ]:
data3 = MonthlyMilkDataset().load()
data3.plot()
train3, test3 = train_test_split(data3, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)

In [ ]:
data4 = HeartRateDataset().load()
data4.plot()
train4, test4 = train_test_split(data4, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)

In [31]:
def visuals(train, test, model):
  model.fit(train)
  pred = model.predict(len(test))
  train.plot(label = 'training')
  test.plot(label = 'testing')
  pred.plot(label = 'prediction')

In [ ]:
print('Air Passeneger Dataset Visualization - pre stationarity tests')
visuals(train1, test1, ARIMA())

In [ ]:
print('Aus Beer Dataset Visualization - pre stationarity tests')
visuals(train2, test2, ARIMA())

In [ ]:
print('Monthly Milk Dataset Visualization - pre stationarity tests')
visuals(train3, test3, ARIMA())

In [ ]:
print('Heart Rate Dataset Visualization - pre stationarity tests')
visuals(train4, test4, ARIMA())

In [81]:
def seasonality1(data):
  for m in range(2, 25):
    is_seasonal, period = check_seasonality(data, m=m, alpha=0.05)
    #print('running {}'.format(m))
    if is_seasonal:
       return period

In [82]:
def seasonality2(data):
  for m in range(2, 25):
    is_seasonal, period = check_seasonality(data, m=m, alpha=0.05)
    #print('running {}'.format(m))
    if is_seasonal:
       return True

In [106]:
def trend(data):
  check = stationarity_tests(data, 0.05, 0.05)
  return not check

In [ ]:
import pandas as pd
stationary = {}
df = pd.DataFrame.from_dict(stationary, orient='index')
col1 = ['dataset', 'air passenger', 'aus beer', 'monthly milk', 'heart rate']
col2 = ['seasonality', seasonality1(data1), seasonality1(data2), seasonality1(data3), seasonality1(data4)]
col3 = ['trend', trend(data1), trend(data2),trend(data3),trend(data4)]
df = pd.DataFrame()
df = pd.DataFrame(col2, col1)
df.insert(0, 1, col3)
print(df)

In [73]:
def vis(trainA, testA, predA):
  print('post removing stationarity')
  trainA.plot(label = 'train')
  testA.plot(label = 'test')
  predA.plot(label = 'pred')

In [133]:
def removeStat(data, train, test):
  if (not stationarity_tests(data, 0.05, 0.05)) and seasonality2(data): #data does have trend and seasonality
    dataT, dataS = extract_trend_and_seasonality(data, freq=seasonality1(data), model=ModelMode.MULTIPLICATIVE, method='naive')
    #either data has trend, seasonality or both
    if (not stationarity_tests(dataT, 0.05, 0.05)) and (seasonality2(dataS)): #both
      dataF = remove_seasonality(data, freq=None, model=SeasonalityMode.ADDITIVE, method='STL')
      dataF = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive')
    elif not stationarity_tests(dataT, 0.05, 0.05): #only trend
      dataF = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive')
    else: #only seasonality
      dataF = remove_seasonality(data, freq=None, model=SeasonalityMode.MULTIPLICATIVE, method='naive')
  else: #data has no trend or seasonality
    dataF = data

  if dataF.is_univariate:
    model = ARIMA()
  else:
    model = VARIMA()
  model.fit(train)
  predF = model.predict(len(test))
  #vis(test, train, predF)

  return dataF

In [ ]:
import pandas as pd
stationary2 = {}
df = pd.DataFrame.from_dict(stationary2, orient='index')
col1 = ['dataset', 'air passenger', 'aus beer', 'monthly milk', 'heart rate']
col2 = ['seasonality', seasonality1(removeStat(data1, train1, test1)), seasonality1(removeStat(data2, train2, test2)), seasonality1(removeStat(data3, train3, test3)), seasonality1(removeStat(data4, train4, test4))]
col3 = ['trend', trend(removeStat(data1, train1, test1)), trend(removeStat(data2, train2, test2)),trend(removeStat(data3, train3, test3)),trend(removeStat(data4, train4, test4))]
df = pd.DataFrame()
df = pd.DataFrame(col2, col1)
df.insert(0, 1, col3)
print(df)

In [143]:
def removeStat2(data, train, test):
  if (not stationarity_tests(data, 0.05, 0.05)) and seasonality2(data): #data does have trend and seasonality
    dataT, dataS = extract_trend_and_seasonality(data, freq=seasonality1(data), model=ModelMode.MULTIPLICATIVE, method='naive')
    #either data has trend, seasonality or both
    if (not stationarity_tests(dataT, 0.05, 0.05)) and (seasonality2(dataS)): #both
      dataF = remove_seasonality(data, freq=None, model=SeasonalityMode.ADDITIVE, method='STL')
      dataF = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive')
    elif not stationarity_tests(dataT, 0.05, 0.05): #only trend
      dataF = remove_trend(data, model=ModelMode.MULTIPLICATIVE, method='naive')
    else: #only seasonality
      dataF = remove_seasonality(data, freq=None, model=SeasonalityMode.MULTIPLICATIVE, method='naive')
  else: #data has no trend or seasonality
    dataF = data

  if dataF.is_univariate:
    model = ARIMA()
  else:
    model = VARIMA()
  model.fit(train)
  predF = model.predict(len(test))
  #vis(test, train, predF)

  return model

In [145]:
def eval_models_mape(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return "{}    {:.2f}%".format(model, mape(testF, pred))

def eval_models_mae(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}    {:.2f}%".format(model, mae(testF, pred)))

def eval_models_cov(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, coefficient_of_variation(testF, pred)))

def eval_models_dtw(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, dtw_metric(testF, pred)))

def eval_models_marre(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, marre(testF, pred)))

def eval_models_mse(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}    {:.2f}%".format(model, mse(testF, pred)))

def eval_models_ope(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, ope(testF, pred)))

def eval_models_r2_score(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, r2_score(testF, pred)))

def eval_models_rho_risk(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}    {:.2f}%".format(model, rho_risk(testF, pred)))
#RHO RISK ONLY WORKS FOR STOCHASTIC TIME SERIES

def eval_models_rmse(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, rmse(testF, pred)))

def eval_models_rmsle(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, rmsle(testF, pred)))

def eval_models_smape(model, data):
  trainF, testF = train_test_split(data, test_size=0.25, axis=0, input_size=0, horizon=0, vertical_split_type='simple', lazy=False)
  model.fit(trainF)
  pred = model.predict(len(testF))
  return("{}   {:.2f}%".format(model, smape(testF, pred)))

#def eval_models_mase(model, train, test):
  #model.fit(train)
  #pred = model.predict(len(test))
  #insample = 
  #return("model {} obtains MASE: {:.2f}%".format(model, mase(test, pred, insample)))

In [ ]:
metrics1 = {}
df = pd.DataFrame.from_dict(metrics1, orient='index')
print('Air Passenger Dataset')
col1 = ['METRICS', 'MAPE', 'COFFECIENT OF VARIATION','DTW METRIC', 'MAE', 'MARRE', 'MASE','MSE', 'OPE', 'R2 SCORE', 'RHO RISK', 'RMSE', 'RMSLE', 'SMAPE']
col2 = ['non stationary', eval_models_mape(ARIMA(), data1), eval_models_cov(ARIMA(), data1), eval_models_dtw(ARIMA(), data1),eval_models_mae(ARIMA(), data1), eval_models_marre(ARIMA(), data1) , "doesnt work", eval_models_mse(ARIMA(), data1), eval_models_ope(ARIMA(), data1), eval_models_r2_score(ARIMA(), data1), "doesnt work", eval_models_rmse(ARIMA(), data1), eval_models_rmsle(ARIMA(), data1), eval_models_smape(ARIMA(), data1)]
col3 = ['stationary', eval_models_mape(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_cov(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_dtw(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_mae(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_marre(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)) , "doesnt work", eval_models_mse(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), 
eval_models_ope(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_r2_score(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), "doesnt work", eval_models_rmse(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_rmsle(removeStat2(data1, train1, test1), removeStat(data1, train1, test1)), eval_models_smape(removeStat2(data1, train1, test1), removeStat(data1, train1, test1))]
df = pd.DataFrame()
df = pd.DataFrame(col2, col1)
df.insert(0, 1, col3)
print(df)

In [149]:
metrics2 = {}
df = pd.DataFrame.from_dict(metrics2, orient='index')
print('Aus Beer Dataset')
col1 = ['METRICS', 'MAPE', 'COFFECIENT OF VARIATION','DTW METRIC', 'MAE', 'MARRE', 'MASE','MSE', 'OPE', 'R2 SCORE', 'RHO RISK', 'RMSE', 'RMSLE', 'SMAPE']
col2 = ['non stationary', eval_models_mape(ARIMA(), data2), eval_models_cov(ARIMA(), data2), eval_models_dtw(ARIMA(), data2),eval_models_mae(ARIMA(), data2), eval_models_marre(ARIMA(), data2) , "doesnt work", eval_models_mse(ARIMA(), data2), eval_models_ope(ARIMA(), data2), eval_models_r2_score(ARIMA(), data2), "doesnt work", eval_models_rmse(ARIMA(), data2), eval_models_rmsle(ARIMA(), data2), eval_models_smape(ARIMA(), data2)]
col3 = ['stationary', eval_models_mape(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), eval_models_cov(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), eval_models_dtw(removeStat2(data2, train2, test2), removeStat(data2, train2, test1)), eval_models_mae(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), eval_models_marre(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)) , "doesnt work", eval_models_mse(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), 
eval_models_ope(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), eval_models_r2_score(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), "doesnt work", eval_models_rmse(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), eval_models_rmsle(removeStat2(data2, train2, test2), removeStat(data2, train2, test2)), eval_models_smape(removeStat2(data2, train2, test2), removeStat(data2, train2, test2))]
df = pd.DataFrame()
df = pd.DataFrame(col2, col1)
df.insert(0, 1, col3)
print(df)

Aus Beer Dataset


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimiz

                                                1                           0
METRICS                                stationary              non stationary
MAPE                     ARIMA(12, 1, 0)    2.32%    ARIMA(12, 1, 0)    3.66%
COFFECIENT OF VARIATION   ARIMA(12, 1, 0)   2.82%     ARIMA(12, 1, 0)   4.54%
DTW METRIC                ARIMA(12, 1, 0)   0.02%    ARIMA(12, 1, 0)   15.33%
MAE                      ARIMA(12, 1, 0)    0.02%   ARIMA(12, 1, 0)    15.70%
MARRE                     ARIMA(12, 1, 0)   6.98%    ARIMA(12, 1, 0)   10.75%
MASE                                  doesnt work                 doesnt work
MSE                      ARIMA(12, 1, 0)    0.00%  ARIMA(12, 1, 0)    384.31%
OPE                       ARIMA(12, 1, 0)   0.09%     ARIMA(12, 1, 0)   3.19%
R2 SCORE                  ARIMA(12, 1, 0)   0.91%     ARIMA(12, 1, 0)   0.79%
RHO RISK                              doesnt work                 doesnt work
RMSE                      ARIMA(12, 1, 0)   0.03%    ARIMA(12, 1

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [148]:
metrics3 = {}
df = pd.DataFrame.from_dict(metrics3, orient='index')
print('Monthly Milk Dataset')
col1 = ['METRICS', 'MAPE', 'COFFECIENT OF VARIATION','DTW METRIC', 'MAE', 'MARRE', 'MASE','MSE', 'OPE', 'R2 SCORE', 'RHO RISK', 'RMSE', 'RMSLE', 'SMAPE']
col2 = ['non stationary', eval_models_mape(ARIMA(), data3), eval_models_cov(ARIMA(), data3), eval_models_dtw(ARIMA(), data1),eval_models_mae(ARIMA(), data3), eval_models_marre(ARIMA(), data3) , "doesnt work", eval_models_mse(ARIMA(), data3), eval_models_ope(ARIMA(), data3), eval_models_r2_score(ARIMA(), data3), "doesnt work", eval_models_rmse(ARIMA(), data3), eval_models_rmsle(ARIMA(), data3), eval_models_smape(ARIMA(), data3)]
col3 = ['stationary', eval_models_mape(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_cov(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_dtw(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_mae(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_marre(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)) , "doesnt work", eval_models_mse(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), 
eval_models_ope(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_r2_score(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), "doesnt work", eval_models_rmse(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_rmsle(removeStat2(data3, train3, test3), removeStat(data3, train3, test3)), eval_models_smape(removeStat2(data3, train3, test3), removeStat(data3, train3, test3))]
df = pd.DataFrame()
df = pd.DataFrame(col2, col1)
df.insert(0, 1, col3)
print(df)

Monthly Milk Dataset


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespac

                                                1                           0
METRICS                                stationary              non stationary
MAPE                     ARIMA(12, 1, 0)    0.93%    ARIMA(12, 1, 0)    2.07%
COFFECIENT OF VARIATION   ARIMA(12, 1, 0)   1.29%     ARIMA(12, 1, 0)   2.43%
DTW METRIC                ARIMA(12, 1, 0)   0.01%    ARIMA(12, 1, 0)   25.16%
MAE                      ARIMA(12, 1, 0)    0.01%   ARIMA(12, 1, 0)    17.10%
MARRE                     ARIMA(12, 1, 0)   3.82%     ARIMA(12, 1, 0)   8.18%
MASE                                  doesnt work                 doesnt work
MSE                      ARIMA(12, 1, 0)    0.00%  ARIMA(12, 1, 0)    427.18%
OPE                       ARIMA(12, 1, 0)   0.14%     ARIMA(12, 1, 0)   1.93%
R2 SCORE                  ARIMA(12, 1, 0)   0.96%     ARIMA(12, 1, 0)   0.88%
RHO RISK                              doesnt work                 doesnt work
RMSE                      ARIMA(12, 1, 0)   0.01%    ARIMA(12, 1

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [147]:
metrics4 = {}
df = pd.DataFrame.from_dict(metrics4, orient='index')
print('Heart rate Dataset')
col1 = ['METRICS', 'MAPE', 'COFFECIENT OF VARIATION','DTW METRIC', 'MAE', 'MARRE', 'MASE','MSE', 'OPE', 'R2 SCORE', 'RHO RISK', 'RMSE', 'RMSLE', 'SMAPE']
col2 = ['non stationary', eval_models_mape(ARIMA(), data4), eval_models_cov(ARIMA(), data4), eval_models_dtw(ARIMA(), data4),eval_models_mae(ARIMA(), data4), eval_models_marre(ARIMA(), data4) , "doesnt work", eval_models_mse(ARIMA(), data4), eval_models_ope(ARIMA(), data4), eval_models_r2_score(ARIMA(), data4), "doesnt work", eval_models_rmse(ARIMA(), data4), eval_models_rmsle(ARIMA(), data4), eval_models_smape(ARIMA(), data4)]
col3 = ['stationary', eval_models_mape(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_cov(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_dtw(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_mae(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_marre(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)) , "doesnt work", eval_models_mse(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), 
eval_models_ope(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_r2_score(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), "doesnt work", eval_models_rmse(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_rmsle(removeStat2(data4, train4, test4), removeStat(data4, train4, test4)), eval_models_smape(removeStat2(data4, train4, test4), removeStat(data4, train4, test4))]
df = pd.DataFrame()
df = pd.DataFrame(col2, col1)
df.insert(0, 1, col3)
print(df)

Heart rate Dataset


/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/stattools.py:2022: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-va

                                                 1                          0
METRICS                                 stationary             non stationary
MAPE                      ARIMA(12, 1, 0)    6.04%   ARIMA(12, 1, 0)    6.04%
COFFECIENT OF VARIATION    ARIMA(12, 1, 0)   7.52%    ARIMA(12, 1, 0)   7.52%
DTW METRIC                 ARIMA(12, 1, 0)   4.81%    ARIMA(12, 1, 0)   4.81%
MAE                       ARIMA(12, 1, 0)    5.72%   ARIMA(12, 1, 0)    5.72%
MARRE                     ARIMA(12, 1, 0)   18.35%   ARIMA(12, 1, 0)   18.35%
MASE                                   doesnt work                doesnt work
MSE                      ARIMA(12, 1, 0)    48.83%  ARIMA(12, 1, 0)    48.83%
OPE                        ARIMA(12, 1, 0)   3.65%    ARIMA(12, 1, 0)   3.65%
R2 SCORE                  ARIMA(12, 1, 0)   -0.31%   ARIMA(12, 1, 0)   -0.31%
RHO RISK                               doesnt work                doesnt work
RMSE                       ARIMA(12, 1, 0)   6.99%    ARIMA(12, 